In [ ]:
%matplotlib inline
import tensorflow as tf
from matplotlib import pyplot as plt
import numpy as np
# easier access to tensorflow distributions
ds = tf.contrib.distributions

An advanced inference tool should support some tweaking and instrospection by the user, therefore a composable design in important. An ideal interface would have high level constructs for the model and the parameters and utilities to compute the most relevant statistical quantities. Further down the line, serialization would be important to share/import/export statistical models.

About statistical models, we can clearly distinguish three different types:
1. Those for which we can analytically evaluate the normalized pdf (e.g. multibin Poisson likelihood, gaussian, bijectable-tranform over a distribribution with and analytical pdf)
2. Those for which we can compute the unnormalized pdf but we do not analytically know the normalization.
3. Those for which we cannot compute even the unnormalized pdf, because they are too complex and/or probabilistc/stochastic.

In HEP, we normally deal with the first type (a realistic statiscal model for each event is of the third type but we cannot deal with that so we use sample summary statistics or very simplified pdf modelling, all of it corresponding to type 1).

I could not think of an example usage of type 2 statistical models in HEP, but in principle are equivalent to type 1 as long as you compute the normalization integral.

Therefore, the initial focus of the tools should be type 1 statistical models. Other tools like Edward/Pyro deal already with general probabilistic models (type 3).

The most relevant function to evaluate over the data for classical inference is the likelihood, which is a function that will depend all model parameters. For many use cases, we have to evaluate this function or its derivatives a many points at once, so being able to vectorize its evaluation would be specially interesting. This leads to the concept of having a batch of parameters. However, quite often optimization (finding maximum likelihood estimate) or constrained optimization (finding maximum likelihood with a fixed set of parameters fixed  is required). Would be worth also to do things like the constrained minimization (with some variables fixed as different values) in a batch to reuse operations and autodiff passes?

For most use cases, data will be kept constant so it would be quite interesting to keep it in a tf.Variable between tf.Session runs and avoid to continuosly feed it via run, this is also what is done for edward when a Python object is passed to the Inference class. What to do with model parameters $\theta$ depend on wether we want to just evaluate likelihood for a set of parameters, find maximum likelihood estimate and its parabolic error or do a full profile likelihood. For optimization, it might be good to be able to keep the parameter values in place in the graph. Then there is the use case of running frequentist toys (where we have to sample the model pdf) and or doing Bayesian inference (were we have to sample the priors and the pdf).